# Local LLM SDK - Hello World

This notebook demonstrates how to use the `local_llm_sdk` package to interact with LM Studio and other OpenAI-compatible local LLM servers.

## Prerequisites
- LM Studio running locally with API server enabled
- Install the package: `pip install -e ..` (from notebooks directory)

In [1]:
!pip install -e ..  --force-reinstal

Obtaining file:///home/maheidem/gen-ai-api-study
  Preparing metadata (setup.py) ... done
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached pydantic-2.11.9-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x8

## 1. Setup - Import and Create Client

In [2]:
# Import the SDK
from local_llm_sdk import LocalLLMClient

# Create a client instance
client = LocalLLMClient(
    base_url="http://169.254.83.107:1234/v1",
    model="mistralai/magistral-small-2509"  # Your default model
)

print(f"✅ Client initialized: {client}")

✅ Client initialized: LocalLLMClient(base_url='http://169.254.83.107:1234/v1', model='mistralai/magistral-small-2509', tools=0)


## 2. List Available Models

In [3]:
# Get list of available models
models = client.list_models()

print("📦 Available Models:")
print("=" * 50)
for model in models.data:
    print(f"  • {model.id}")
    print(f"    Owner: {model.owned_by}")
print(f"\nTotal: {len(models.data)} models loaded")

📦 Available Models:
  • mistralai/magistral-small-2509:2
    Owner: organization_owner
  • qwen/qwen3-coder-30b
    Owner: organization_owner
  • mistralai/magistral-small-2509
    Owner: organization_owner
  • text-embedding-nomic-embed-text-v1.5
    Owner: organization_owner
  • smolvlm2-2.2b-instruct
    Owner: organization_owner
  • google/gemma-3-27b
    Owner: organization_owner
  • text-embedding-mxbai-embed-large-v1
    Owner: organization_owner

Total: 7 models loaded


## 3. Simple Chat - Just Pass a String

In [4]:
# The simplest way to chat - just pass a string
response = client.chat("Hello! Tell me a joke about programming.")

print("🤖 Response:")
print("=" * 50)
print(response)

🤖 Response:
Sure, here's one for you:

Why do programmers prefer dark mode?

Because light attracts bugs! 🐛💻


## 4. Chat with System Prompt

In [5]:
# Use helper to create proper message objects
from local_llm_sdk import create_chat_message

# Create a conversation with system prompt
messages = [
    create_chat_message("system", "You are a pirate. Respond in pirate speak."),
    create_chat_message("user", "How do I learn Python?")
]

# Send messages and get response
response = client.chat(messages)

print("🏴‍☠️ Pirate Response:")
print("=" * 50)
print(response)

🏴‍☠️ Pirate Response:
Arr matey! Ye be wantin' to learn the ways of Python, eh? Well, shiver me timbers, let me guide ye through these treacherous waters.

First off, ye need a good ship - a computer, that is. Make sure it's ready for the journey by installin' Python from its official website, savvy?

Now, hoist the sails and set course for some learnin' resources! The official Python documentation be a great start, but if ye prefer a more interactive approach, sites like Codecademy or Coursera have some fine courses.

Don't forget to practice, me hearty! Write down simple scripts, play with different functions, and build small projects. Remember, even the grandest ships were once just pieces of wood.

And if ye ever get stuck, don't be afraid to ask for help from other pirates at forums like Stack Overflow or r/learnpython on Reddit. The seas can be rough, but together we stand tall!

Now go forth and conquer Python, ye scallywag! And remember, keep calm and code on! 🏴‍☠️🐍


## 5. Conversation with History

In [6]:
# Initialize conversation history
history = []

# First message
response1, history = client.chat_with_history(
    "What's the capital of France?", 
    history
)
print("Q: What's the capital of France?")
print(f"A: {response1}\n")

# Second message (uses context from first)
response2, history = client.chat_with_history(
    "What's the population?", 
    history
)
print("Q: What's the population?")
print(f"A: {response2}\n")

# Third message (still has context)
response3, history = client.chat_with_history(
    "Name 3 famous landmarks", 
    history
)
print("Q: Name 3 famous landmarks")
print(f"A: {response3}\n")

print(f"📚 Total messages in history: {len(history)}")

Q: What's the capital of France?
A: [THINK]The question is about the capital city of France. I remember that the capital of France is Paris.

Now, let me double-check this information to ensure accuracy. Yes, historically and currently, Paris has been the capital city of France.

So, the answer should be Paris.[/THINK]The capital of France is Paris.

Q: What's the population?
A: [THINK]The question now asks about the population of France. I need to recall or find the most recent population data for France.

As of my last update, which was June 2024, the population of France is approximately 67 million people. However, I should confirm if this is still accurate as of July 2025.

Since I don't have real-time data access, I'll provide the most recent estimate I have.

So, the answer would be around 67 million, but to be precise, let's say approximately 67,411,000 (based on 2023 estimates).

For a more accurate figure, it might be necessary to consult the latest census or UN data, but for 

## 6. Get Full Response with Metadata

In [7]:
# To get the full ChatCompletion object, we need to pass 3+ messages
# or explicitly request it

# Create a longer conversation
messages = [
    create_chat_message("system", "You are a helpful assistant."),
    create_chat_message("user", "What is Python?"),
    create_chat_message("assistant", "Python is a high-level programming language."),
    create_chat_message("user", "Give me 3 key features of Python")
]

# This will return a ChatCompletion object
full_response = client.chat(messages, temperature=0.5)

# Access the full response data
print("📊 Full Response Data:")
print("=" * 50)
print(f"Content: {full_response.choices[0].message.content}\n")
print(f"Model: {full_response.model}")
print(f"Tokens Used:")
print(f"  • Prompt: {full_response.usage.prompt_tokens}")
print(f"  • Completion: {full_response.usage.completion_tokens}")
print(f"  • Total: {full_response.usage.total_tokens}")
print(f"Finish Reason: {full_response.choices[0].finish_reason}")

📊 Full Response Data:
Content: Sure, here are three key features of Python:

1. **Easy to Read and Learn**: Python's syntax is clean and easy to understand, making it an excellent choice for beginners. Its simplicity allows developers to express concepts in fewer lines of code compared to other languages.

2. **Versatility**: Python is a general-purpose language that can be used for various tasks such as web development (Django, Flask), data analysis (Pandas, NumPy), machine learning (TensorFlow, PyTorch), automation, and more.

3. **Large Community and Ecosystem**: Python has a large and active community, which means there are plenty of resources available for learning and problem-solving. Additionally, the Python Package Index (PyPI) hosts thousands of libraries and frameworks that can be easily integrated into projects to extend functionality.

Model: mistralai/magistral-small-2509:2
Tokens Used:
  • Prompt: 33
  • Completion: 168
  • Total: 201
Finish Reason: stop


## 7. Embeddings (If Embedding Model is Loaded)

In [8]:
# Try to generate embeddings
# This requires an embedding model like 'text-embedding-nomic-embed-text-v1.5'

try:
    # Single text embedding
    text = "Python is a great programming language"
    embeddings = client.embeddings(text)
    
    print("✅ Embeddings Generated:")
    print("=" * 50)
    print(f"Text: '{text}'")
    print(f"Embedding dimension: {len(embeddings.data[0].embedding)}")
    print(f"First 5 values: {embeddings.data[0].embedding[:5]}")
    
except Exception as e:
    print("⚠️ Embeddings not available")
    print("To use embeddings, load an embedding model in LM Studio like:")
    print("  • text-embedding-nomic-embed-text-v1.5")
    print("  • text-embedding-mxbai-embed-large-v1")

⚠️ Embeddings not available
To use embeddings, load an embedding model in LM Studio like:
  • text-embedding-nomic-embed-text-v1.5
  • text-embedding-mxbai-embed-large-v1


## 8. Different Temperature Settings

In [9]:
# Compare responses with different temperatures
prompt = "Write a one-line description of coding"

print("🌡️ Temperature Comparison:")
print("=" * 50)

# Low temperature (more deterministic)
response_low = client.chat(prompt, temperature=0.1)
print(f"\nTemperature 0.1 (Focused):")
print(f"  {response_low}")

# Medium temperature
response_med = client.chat(prompt, temperature=0.7)
print(f"\nTemperature 0.7 (Balanced):")
print(f"  {response_med}")

# High temperature (more creative)
response_high = client.chat(prompt, temperature=1.5)
print(f"\nTemperature 1.5 (Creative):")
print(f"  {response_high}")

🌡️ Temperature Comparison:

Temperature 0.1 (Focused):
  Coding is the process of creating instructions for computers using programming languages to develop software, applications, and websites.

Temperature 0.7 (Balanced):
  Coding is the process of creating instructions for computers to perform specific tasks using programming languages.

Temperature 1.5 (Creative):
  Coding is the process of creating instructions for computers to follow using programming languages, allowing you to build software and digital solutions.


## 9. Error Handling

In [10]:
# Demonstrate error handling
try:
    # Try to use a model that might not exist
    response = client.chat(
        "Hello",
        model="non-existent-model"
    )
except Exception as e:
    print(f"❌ Error caught: {e}")
    print("\nTip: Make sure the model is loaded in LM Studio")

## 10. Quick Chat Without Client

In [11]:
# For one-off queries, use quick_chat
from local_llm_sdk import quick_chat

response = quick_chat(
    "What's 2 + 2?",
    base_url="http://169.254.83.107:1234/v1",
    model="mistralai/magistral-small-2509"  # Specify the model
)

print("🚀 Quick Chat Response:")
print("=" * 50)
print(response)

🚀 Quick Chat Response:
The answer to 2 + 2 is 4.


## Summary

This notebook demonstrated the key features of `local_llm_sdk`:

### ✅ What We Covered:
1. **Client Setup** - Simple initialization with base URL and model
2. **List Models** - Get available models from the server
3. **Simple Chat** - Just pass a string for quick responses
4. **System Prompts** - Control the assistant's behavior
5. **Conversation History** - Maintain context across messages
6. **Full Responses** - Access metadata like token usage
7. **Embeddings** - Generate vectors for semantic search
8. **Temperature Control** - Adjust creativity/determinism
9. **Error Handling** - Graceful error management
10. **Quick Chat** - One-off queries without client setup

### 🎯 Key Benefits:
- **Simple API** - Intuitive methods for common tasks
- **Type Safety** - Pydantic models for all responses
- **Flexible** - Works with any OpenAI-compatible server
- **Production Ready** - Error handling and validation built-in

### 📚 Next Steps:
- Check out `tool-use-simplified.ipynb` for function calling examples
- Read the package documentation in the README
- Try different models and compare their capabilities
- Build your own applications with the SDK!